In [126]:
import pandas as pd

# Superstore

In [127]:
path ="../data/sample_superstore.csv"
df = pd.read_csv(path, encoding='latin1')

In [128]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [129]:
df.columns.tolist()

['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

In [130]:
neccessary_columns = [
    'Order ID', 'Order Date', 'Customer ID',  'Sales'
]

df_filtered = df[neccessary_columns].copy()

print(df_filtered.shape)
df_filtered.describe(include='all').round(2)

(9994, 4)


,Order ID,Order Date,Customer ID,Sales
count,9994,9994,9994,9994.00
unique,5009,1237,793,NaN
top,CA-2017-100111,9/5/2016,WB-21850,NaN
freq,14,38,37,NaN
mean,NaN,NaN,NaN,229.86
std,NaN,NaN,NaN,623.25
min,NaN,NaN,NaN,0.44
25%,NaN,NaN,NaN,17.28
50%,NaN,NaN,NaN,54.49
75%,NaN,NaN,NaN,209.94


In [131]:
column_mapping = {
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Customer ID': 'customer_id',
    'Sales': 'sales_amount'
}

df_filtered.rename(columns=column_mapping, inplace=True)

df_filtered = df_filtered.groupby(['order_id', 'order_date', 'customer_id'])\
    .agg({'sales_amount': 'sum'})\
        .reset_index()

Data Type checks

In [132]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      5009 non-null   object 
 1   order_date    5009 non-null   object 
 2   customer_id   5009 non-null   object 
 3   sales_amount  5009 non-null   float64
dtypes: float64(1), object(3)
memory usage: 156.7+ KB


In [133]:
# Data Type Fixes
df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])
df_filtered['order_id'] = df_filtered['order_id'].str.strip().astype('string')
df_filtered['customer_id'] = df_filtered['customer_id'].str.strip().astype('string')
df_filtered['sales_amount'] = pd.to_numeric(df_filtered['sales_amount'], errors='coerce')

df_filtered.dtypes

order_id        string[python]
order_date      datetime64[ns]
customer_id     string[python]
sales_amount           float64
dtype: object

Sanity Check of Data

date

In [134]:
df_filtered['order_date'].describe()

count                             5009
mean     2016-05-07 16:04:47.482531328
min                2014-01-03 00:00:00
25%                2015-05-31 00:00:00
50%                2016-07-10 00:00:00
75%                2017-05-19 00:00:00
max                2017-12-30 00:00:00
Name: order_date, dtype: object

In [135]:
# time = pd.DataFrame()
# time['date'] = pd.date_range(start=df_filtered['order_date'].min(), end=df_filtered['order_date'].max(), freq='D')
# time.describe()

sales amount

In [136]:
df_filtered['sales_amount'].min(), df_filtered['sales_amount'].max()

df_filtered['sales_amount'].describe()

count     5009.000000
mean       458.614666
std        954.730953
min          0.556000
25%         37.630000
50%        151.960000
75%        512.064000
max      23661.228000
Name: sales_amount, dtype: float64

order id

In [137]:
(df_filtered['order_id'].value_counts().sort_values(ascending=False) > 1).sum()

np.int64(0)

customer id

In [138]:
df_filtered['customer_id'].value_counts().head()

customer_id
EP-13915    17
CK-12205    13
SH-19975    13
JE-15745    13
NS-18640    13
Name: count, dtype: Int64

In [139]:
df_filtered.head()
df_filtered.to_excel("../data/cleaned_sample_superstore.xlsx", index=False)

# Retail

In [140]:
df = pd.read_excel("../data/Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [141]:
df.columns.tolist()

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [142]:
df['Sales'] = df['Quantity'] * df['UnitPrice']

neccessary_columns = [
    'InvoiceNo', 'InvoiceDate', 'CustomerID',  'Sales'
]

df_filtered = df[neccessary_columns].copy()

print(df_filtered.shape)
df_filtered.describe(include='all').round(2)

(541909, 4)


,InvoiceNo,InvoiceDate,CustomerID,Sales
count,541909.0,541909,406829.00,541909.00
unique,25900.0,NaN,NaN,NaN
top,573585.0,NaN,NaN,NaN
freq,1114.0,NaN,NaN,NaN
mean,NaN,2011-07-04 13:34:57.156386048,15287.69,17.99
min,NaN,2010-12-01 08:26:00,12346.00,-168469.60
25%,NaN,2011-03-28 11:34:00,13953.00,3.40
50%,NaN,2011-07-19 17:17:00,15152.00,9.75
75%,NaN,2011-10-19 11:27:00,16791.00,17.40
max,NaN,2011-12-09 12:50:00,18287.00,168469.60


In [143]:
column_mapping = {
    'InvoiceNo': 'order_id',
    'InvoiceDate': 'order_date',
    'CustomerID': 'customer_id',
    'Sales': 'sales_amount'
}

df_filtered.rename(columns=column_mapping, inplace=True)

df_filtered = df_filtered.groupby(['order_id', 'order_date', 'customer_id'])\
    .agg({'sales_amount': 'sum'})\
        .reset_index()

Data Type checks

In [144]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22221 entries, 0 to 22220
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      22221 non-null  object        
 1   order_date    22221 non-null  datetime64[ns]
 2   customer_id   22221 non-null  float64       
 3   sales_amount  22221 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 694.5+ KB


In [145]:
# Data Type Fixes
df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])
df_filtered['order_id'] = df_filtered['order_id'].str.strip().astype('string')
df_filtered['customer_id'] = df_filtered['customer_id'].astype('string')
df_filtered['sales_amount'] = pd.to_numeric(df_filtered['sales_amount'], errors='coerce')

df_filtered.dtypes

order_id        string[python]
order_date      datetime64[ns]
customer_id     string[python]
sales_amount           float64
dtype: object

Sanity Check of Data

date

In [146]:
df_filtered['order_date'].describe()

count                            22221
mean     2011-06-29 16:42:17.402457088
min                2010-12-01 08:26:00
25%                2011-03-28 15:42:00
50%                2011-07-08 12:22:00
75%                2011-10-10 17:11:00
max                2011-12-09 12:50:00
Name: order_date, dtype: object

sales amount

In [147]:
df_filtered['sales_amount'].min(), df_filtered['sales_amount'].max()

df_filtered['sales_amount'].describe()

count     22221.000000
mean        373.523505
std        2016.999501
min     -168469.600000
25%          87.440000
50%         240.680000
75%         417.620000
max      168469.600000
Name: sales_amount, dtype: float64

In [148]:
df['Sales'].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.95, 0.99, 0.995])
percent1 = df['Sales'].quantile(0.1)
percent99 = df['Sales'].quantile(0.99)

df_filtered = df_filtered[(df_filtered['sales_amount'] >= percent1) & (df_filtered['sales_amount'] <= percent99)]

In [149]:
df_filtered.sales_amount.describe()

count    5464.000000
mean      102.564343
std        50.900727
min         1.650000
25%        62.400000
50%       111.990000
75%       144.582500
max       180.000000
Name: sales_amount, dtype: float64

order id

In [150]:
(df_filtered['order_id'].value_counts().sort_values(ascending=False) > 1).sum()

np.int64(0)

customer id

In [151]:
df_filtered['customer_id'].value_counts().head()

customer_id
12748.0    147
14606.0     83
12971.0     70
14527.0     39
17961.0     38
Name: count, dtype: Int64

In [152]:
df_filtered.head()
df_filtered.to_excel("../data/cleaned_sample_retail.xlsx", index=False)

# Sales_Product_Combined.csv

In [153]:
df = pd.read_csv("../data/Sales_Product_Combined.csv")
df.head()

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones


In [154]:
fake_user_id = pd.DataFrame({ 
        'address':
            df['Purchase Address'].unique(), 
        'user_id':
            range(1, len(df['Purchase Address'].unique())+1)
        }
        )

df = df.merge(fake_user_id, left_on='Purchase Address', right_on='address', how='left')

In [155]:
df.columns.tolist()

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price',
 'Order Date',
 'Time',
 'Purchase Address',
 'City',
 'Product Type',
 'address',
 'user_id']

In [156]:
df['Sales'] = df['Quantity Ordered'] * df['Price']

neccessary_columns = [
    'Order ID', 'Order Date', 'user_id',  'Sales'
]

df_filtered = df[neccessary_columns].copy()

print(df_filtered.shape)
df_filtered.describe(include='all').round(2)

(185950, 4)


,Order ID,Order Date,user_id,Sales
count,185950.00,185950,185950.00,185950
unique,NaN,366,NaN,56
top,NaN,17-12-2019,NaN,14.95
freq,NaN,881,NaN,20210
mean,230417.57,NaN,65628.92,NaN
std,51512.74,NaN,40282.25,NaN
min,141234.00,NaN,1.00,NaN
25%,185831.25,NaN,30439.00,NaN
50%,230367.50,NaN,63380.50,NaN
75%,275035.75,NaN,99549.75,NaN


In [157]:
column_mapping = {
    neccessary_columns[0]: 'order_id',
    neccessary_columns[1]: 'order_date',
    neccessary_columns[2]: 'customer_id',
    neccessary_columns[3]: 'sales_amount'
}

df_filtered.rename(columns=column_mapping, inplace=True)

df_filtered = df_filtered.groupby(['order_id', 'order_date', 'customer_id'])\
    .agg({'sales_amount': 'sum'})\
        .reset_index()

Data Type checks

In [158]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178437 entries, 0 to 178436
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      178437 non-null  int64 
 1   order_date    178437 non-null  object
 2   customer_id   178437 non-null  int64 
 3   sales_amount  178437 non-null  object
dtypes: int64(2), object(2)
memory usage: 5.4+ MB


In [159]:
# Data Type Fixes
df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])
df_filtered['order_id'] = df_filtered['order_id'].astype('string')
df_filtered['customer_id'] = df_filtered['customer_id'].astype('string')
df_filtered['sales_amount'] = pd.to_numeric(df_filtered['sales_amount'], errors='coerce')

df_filtered.dtypes

/var/folders/kh/21c5gvns57q5bq666b4_v4hh0000gn/T/ipykernel_87419/74203057.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])


order_id        string[python]
order_date      datetime64[ns]
customer_id     string[python]
sales_amount           float64
dtype: object

Sanity Check of Data

date

In [160]:
df_filtered['order_date'].describe()

count                           178437
mean     2019-07-18 08:41:57.379243008
min                2019-01-01 00:00:00
25%                2019-04-16 00:00:00
50%                2019-07-18 00:00:00
75%                2019-10-26 00:00:00
max                2020-01-01 00:00:00
Name: order_date, dtype: object

sales amount

In [161]:
df_filtered['sales_amount'].min(), df_filtered['sales_amount'].max()

df_filtered['sales_amount'].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.995])

count    1.545650e+05
mean     4.727182e+05
std      1.780998e+08
min      2.990000e+00
1%       2.990000e+00
5%       2.990000e+00
10%      3.840000e+00
25%      1.195000e+01
50%      1.495000e+01
75%      1.500000e+02
90%      6.000000e+02
95%      9.999900e+02
99%      7.001495e+04
99.5%    7.001495e+04
max      7.001502e+10
Name: sales_amount, dtype: float64

In [162]:
percent1 = df_filtered['sales_amount'].min()
percent99 = df_filtered['sales_amount'].quantile(0.9)

df_filtered = df_filtered[(df_filtered['sales_amount'] >= percent1) & (df_filtered['sales_amount'] <= percent99)]

In [163]:
df_filtered.sales_amount.describe()

count    141780.000000
mean        108.251024
std         151.971478
min           2.990000
25%          11.950000
50%          14.950000
75%         150.000000
max         600.000000
Name: sales_amount, dtype: float64

order id

In [164]:
(df_filtered['order_id'].value_counts().sort_values(ascending=False) > 1).sum()

np.int64(0)

customer id

In [165]:
df_filtered['customer_id'].value_counts().head()

customer_id
4709     6
33998    6
21591    6
11522    6
12445    6
Name: count, dtype: Int64

In [166]:
df_filtered.head()
df_filtered.to_excel("../data/cleaned_sample_sales_combined.xlsx", index=False)